# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/datasets/camnugent/california-housing-prices

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает медианную стоимость домовладения на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [4]:
import numpy as np
import torch, csv

def gen_ser(lst):
    # Создадим итератор при помощи генератора.
    for item in lst:
        yield item

In [5]:
#Создадим класс датасета:

class RangeDataset(torch.utils.data.Dataset):
    def __init__(self, file, start, end, step=1): #нумерация с 1
        self.start = start
        self.end = end
        self.step = step
    
        self.file = file
        self.len = -1
        self.gen = gen_ser([x for x in range(start, end+1, step)]) #все нужные нам индексы
        self.header = None #шапка
        self.cur_ind = 0 #текущий номер строки - используется только при чтении
        
        #создаем текущий reader csv-файла
        csvfile = open(self.file, newline='')
        self.reader = csv.reader(csvfile, delimiter=',') #НАСТРОЙКИ!
        #csvfile.close()

    def __len__(self):
        if self.len == -1:
            i = 0
            while True:
                try:
                    if i==0: #первая строка
                        self.header = self.reader.__next__() #записываем шапку
                    else:
                        self.reader.__next__()
                    
                    i+=1
                except:
                    break
                
            self.len = i - 1 #не учитываем "шапку"
            
            #заново создаем текущий reader csv-файла
            csvfile = open(self.file, newline='')
            self.reader = csv.reader(csvfile, delimiter=',') 
            #csvfile.close()
            
        else:
            pass
        
        return self.len

    def __getitem__(self):
        try:
            indx = next(self.gen) #берем очередной индекс
        except:
            return
        
        while self.cur_ind < indx:
            try:
                value = self.reader.__next__()
                self.cur_ind+=1
            except:                
                #заново создаем текущий reader csv-файла
                csvfile = open(self.file, newline='')
                self.reader = csv.reader(csvfile, delimiter=',') 
                
                print('RangeDataset.__getitem__: ПРЕДУПРЕЖДЕНИЕ: достигнут конец файла')
                self.gen = gen_ser([x for x in range(self.start, self.end+1, self.step)]) #все нужные нам индексы
                
                return
                
        return value

In [6]:
dataset = RangeDataset('housing.csv', 1, 500, 2)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2, drop_last=True)

for i, batch in enumerate(data_loader):
    print(i, batch)

TypeError: cannot pickle 'generator' object

Ничего не получается, поэтому загружу датасет привычным способом: с помощью Pandas

In [19]:
import pandas as pd

data = pd.read_csv('housing.csv', delimiter=',')
print(len(data))

20640


In [20]:
data.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


Проверим датафрейм на nan-значения:

In [21]:
data.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

В поле total_bedrooms имеется 207 пропусков. ЗАполним их медианным значением:

In [22]:
med = data.total_bedrooms.median()
data.total_bedrooms.fillna(med,inplace=True)

Смотрим результаты теперь:

In [23]:
data.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Обратотаем поля следующим образом: вещественные - с помощью StandardScaler; категориальный признак ocean_proximity уберем вообще. Предсказывать будем median_house_value - медианную стоимость дома

In [24]:
from sklearn.preprocessing import StandardScaler

Z_longitude = StandardScaler()
Z_latitude = StandardScaler()
Z_housing_median_age = StandardScaler()
Z_total_rooms = StandardScaler()
Z_total_bedrooms = StandardScaler()
Z_population = StandardScaler()
Z_households = StandardScaler()
Z_median_income = StandardScaler()

#обработаем вещественные признаки
data.longitude = Z_longitude.fit_transform(np.expand_dims(data.longitude.to_numpy(),axis=1))
data.latitude = Z_latitude.fit_transform(np.expand_dims(data.latitude.to_numpy(),axis=1))
data.housing_median_age = Z_housing_median_age.fit_transform(np.expand_dims(data.housing_median_age.to_numpy(),axis=1))
data.total_rooms = Z_total_rooms.fit_transform(np.expand_dims(data.total_rooms.to_numpy(),axis=1))
data.total_bedrooms = Z_total_bedrooms.fit_transform(np.expand_dims(data.total_bedrooms.to_numpy(),axis=1))
data.population = Z_population.fit_transform(np.expand_dims(data.population.to_numpy(),axis=1))
data.households = Z_households.fit_transform(np.expand_dims(data.households.to_numpy(),axis=1))
data.median_income = Z_median_income.fit_transform(np.expand_dims(data.median_income.to_numpy(),axis=1))

#перемешаем данные
data = pd.get_dummies(data,columns=['ocean_proximity'])
data = data.sample(frac=1)

#смотрим итоговый датафрейм
data.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
5885,0.613783,-0.689106,1.379433,-0.568291,-0.579040,-0.721875,-0.663160,-0.094048,244500.0,1,0,0,0,0
18354,-1.267940,0.809091,0.425936,-0.486240,-0.843716,-0.807532,-0.835790,4.903752,500001.0,0,0,0,1,0
8082,0.683662,-0.852971,1.141059,0.172921,0.088609,0.087884,0.077057,0.568135,194000.0,0,0,0,0,1
4868,0.643731,-0.740606,1.220517,-0.624673,-0.149837,0.290103,-0.103420,-1.193284,275000.0,1,0,0,0,0
9449,-0.139904,0.823137,-1.163225,0.049614,0.043305,-0.240611,-0.066802,-0.588317,111100.0,0,1,0,0,0
14371,1.162827,-1.367976,0.267020,0.700524,0.391436,-0.131113,0.281074,1.471452,500001.0,0,0,0,0,1
11325,0.803453,-0.876380,-0.527561,-0.053064,0.045689,0.618599,0.071825,-0.051201,193900.0,1,0,0,0,0
11164,0.783488,-0.848289,-0.845393,0.142667,0.052842,0.348385,0.134600,0.346740,224800.0,1,0,0,0,0
13334,0.948201,-0.740606,-1.798890,0.090411,0.217370,0.257431,0.233993,-0.349763,167500.0,0,1,0,0,0
12210,1.302584,-0.941927,-1.639974,-0.748437,-0.881867,-0.916147,-0.940414,0.221620,221900.0,0,1,0,0,0


Делим выборку на обучающую и тестовую

In [25]:
from sklearn.model_selection import train_test_split

X = data.drop(['median_house_value'],axis=1).to_numpy()
y = np.expand_dims(data.median_house_value.to_numpy(), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(15480, 13) (5160, 13) (15480, 1) (5160, 1)


In [26]:
X.max()

30.250330218731502

Переводим все массивы в тензоры

In [27]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

Создадим 6-слойный перцептрон

In [50]:
from tqdm import tnrange

import torch.nn.functional as fnc, torch.nn as nn
import torch.optim as opt

#6-слойный перцептрон c ФА ReLU
class Model_6l(nn.Module):
    def __init__(self,lays,optim):
        super(Model_6l, self).__init__()
        self.bn1 = nn.BatchNorm1d(lays[0])
        self.lay12=nn.Linear(lays[0],lays[1]).to(device)
        self.bn2 = nn.BatchNorm1d(lays[1])
        self.dp2 = nn.Dropout(0.2)
        self.lay23=nn.Linear(lays[1],lays[2]).to(device)
        self.bn3 = nn.BatchNorm1d(lays[2])
        self.dp3 = nn.Dropout(0.2)
        self.lay34=nn.Linear(lays[2],lays[3]).to(device)
        self.bn4 = nn.BatchNorm1d(lays[3])
        self.lay45=nn.Linear(lays[3],lays[4]).to(device)
        self.bn5 = nn.BatchNorm1d(lays[4])
        self.lay56=nn.Linear(lays[4],lays[5]).to(device)
        
        if optim == 'SGD': #оптимизаторы
            self.opt=opt.SGD(self.parameters(), lr=0.0000005, momentum=0.9) 
        elif optim == 'Adam':
            self.opt=opt.Adam(self.parameters(), lr=0.0001, betas=(0.9, 0.999))
        elif optim == 'RMSprop':
            self.opt=opt.RMSprop(self.parameters(), lr=0.0001, momentum=0.9)
        
        self.criterion=nn.MSELoss() #функция потерь для задачи регрессии

    def forward(self, x):
        x = self.bn1(x)
        x = fnc.relu(self.lay12(x))
        x = self.bn2(x)
        x = self.dp2(x)
        x = fnc.relu(self.lay23(x))
        x = self.bn3(x)
        x = self.dp3(x)
        x = fnc.relu(self.lay34(x))
        x = self.bn4(x)
        x = fnc.relu(self.lay45(x))
        x = self.bn5(x)
        return fnc.relu(self.lay56(x))

In [51]:
#подготовка обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

size_nn = [13, 256, 160, 80, 48, 1] #зафиксируем размер сети
optims = ['SGD', 'Adam', 'RMSprop']

errs_6l = []
mdls_6l = []

Будем обучать каждую сеть на протяжении 1000 эпох

In [53]:
for optim in optims:
    net = Model_6l(size_nn, optim)
    
    #определим размер пакета (в зависимости от оптимизатора)
    batch_size = 1024 # if optim == 'SGD' else 4
    smpls_train = len(y_train)
    batches = int(np.ceil(smpls_train/batch_size))
    
    ep_count = 1000 # if optim == 'SGD' else 256000
    
    for epoch in tnrange(ep_count, desc = 'Сеть с оптимизатором {}'.format(optim)):  
        #running_loss = 0.0
        for i in range(batches):
            if i+1 != batches:
                inputs = X_train[i*batch_size:(i+1)*batch_size, :]
                labels = y_train[i*batch_size:(i+1)*batch_size, :]
            else:
                inputs1 = X_train[i*batch_size:smpls_train, :]
                labels1 = y_train[i*batch_size:smpls_train, :]
                
                resid = batch_size - (smpls_train - i*batch_size)
                    
                inputs2 = X_train[0:resid, :]
                labels2 = y_train[0:resid, :]
                    
                inputs = torch.cat((inputs1,inputs2))
                labels = torch.cat((labels1,labels2))
                
                del inputs1, labels1, inputs2, labels2

            # обнуляем градиент
            net.opt.zero_grad()

            outputs = net(inputs)
            loss = net.criterion(outputs, labels)
            loss.backward()
            net.opt.step()

            # выводим статистику о процессе обучения
            '''
            running_loss += loss.item()
            
            if i % 100 == 0:    # печатаем каждые 100 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000)) # откуда 2000?
                running_loss = 0.0
            
            if epoch % 20 if optim == 'SGD' else 6400 == 0:
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000)) # откуда 2000?
            
            running_loss = 0.0
            '''
            
        
        #вычисление ошибки на тестовом наборе данных
        y_pred = net(X_test)
        loss_test = net.criterion(y_pred, y_test).item()
        
        if epoch % (20) == 0:
            print('[%d] MSE loss: %.3f' % (epoch + 1, loss_test))
        
    
    errs_6l.append(loss_test)
    mdls_6l.append(net)

<ipython-input-53-7d7589076410>:11: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for epoch in tnrange(ep_count, desc = 'Сеть с оптимизатором {}'.format(optim)):


[1] MSE loss: 7469752832.000
[21] MSE loss: 3542206464.000
[41] MSE loss: 3251213568.000
[61] MSE loss: 3058737920.000
[81] MSE loss: 3003651328.000
[101] MSE loss: 3006922240.000
[121] MSE loss: 3172524800.000
[141] MSE loss: 3082678016.000
[161] MSE loss: 2921291776.000
[181] MSE loss: 3174039296.000
[201] MSE loss: 3045385728.000
[221] MSE loss: 2965918464.000
[241] MSE loss: 2892480768.000
[261] MSE loss: 2987486976.000
[281] MSE loss: 2902543616.000
[301] MSE loss: 2741195776.000
[321] MSE loss: 2812776448.000
[341] MSE loss: 2880015104.000
[361] MSE loss: 2727960576.000
[381] MSE loss: 2889634816.000
[401] MSE loss: 2743688704.000
[421] MSE loss: 2857442560.000
[441] MSE loss: 2746368000.000
[461] MSE loss: 2810663424.000
[481] MSE loss: 2809225472.000
[501] MSE loss: 2944125440.000
[521] MSE loss: 2894916864.000
[541] MSE loss: 2851687680.000
[561] MSE loss: 3053588992.000
[581] MSE loss: 2854410752.000
[601] MSE loss: 2933094912.000
[621] MSE loss: 2875321856.000
[641] MSE loss

[1] MSE loss: 56046215168.000
[21] MSE loss: 56044789760.000
[41] MSE loss: 56043888640.000
[61] MSE loss: 56043028480.000
[81] MSE loss: 56042090496.000
[101] MSE loss: 56041082880.000
[121] MSE loss: 56039985152.000
[141] MSE loss: 56038776832.000
[161] MSE loss: 56037449728.000
[181] MSE loss: 56036052992.000
[201] MSE loss: 56034574336.000
[221] MSE loss: 56033026048.000
[241] MSE loss: 56031416320.000
[261] MSE loss: 56029724672.000
[281] MSE loss: 56027987968.000
[301] MSE loss: 56026148864.000
[321] MSE loss: 56024227840.000
[341] MSE loss: 56022257664.000
[361] MSE loss: 56020230144.000
[381] MSE loss: 56018124800.000
[401] MSE loss: 56015962112.000
[421] MSE loss: 56013754368.000
[441] MSE loss: 56011411456.000
[461] MSE loss: 56009043968.000
[481] MSE loss: 56006582272.000
[501] MSE loss: 56004104192.000
[521] MSE loss: 56001540096.000
[541] MSE loss: 55998902272.000
[561] MSE loss: 55996207104.000
[581] MSE loss: 55993430016.000
[601] MSE loss: 55990583296.000
[621] MSE loss

[1] MSE loss: 56044691456.000
[21] MSE loss: 56027422720.000
[41] MSE loss: 56005746688.000
[61] MSE loss: 55977521152.000
[81] MSE loss: 55942819840.000
[101] MSE loss: 55901474816.000
[121] MSE loss: 55853854720.000
[141] MSE loss: 55799877632.000
[161] MSE loss: 55739232256.000
[181] MSE loss: 55672700928.000
[201] MSE loss: 55599022080.000
[221] MSE loss: 55520215040.000
[241] MSE loss: 55434788864.000
[261] MSE loss: 55343484928.000
[281] MSE loss: 55245303808.000
[301] MSE loss: 55141871616.000
[321] MSE loss: 55031910400.000
[341] MSE loss: 54914973696.000
[361] MSE loss: 54793482240.000
[381] MSE loss: 54664781824.000
[401] MSE loss: 54532616192.000
[421] MSE loss: 54394019840.000
[441] MSE loss: 54246154240.000
[461] MSE loss: 54095020032.000
[481] MSE loss: 53935063040.000
[501] MSE loss: 53772451840.000
[521] MSE loss: 53604052992.000
[541] MSE loss: 53427093504.000
[561] MSE loss: 53249712128.000
[581] MSE loss: 53057286144.000
[601] MSE loss: 52871921664.000
[621] MSE loss

Выведем значения ошибки на 1000-ю эпоху для всех сетей:

In [54]:
print(optims)
print(errs_6l)

['SGD', 'Adam', 'RMSprop']
[2994697984.0, 55920254976.0, 47926878208.0]


На 1000 эпохах обучения при размере пакета 1024 и коэффициентах обучения 5e-07 у SGD и 0.0001 у остальных оптимизаторов (в 200 раз больше) быстрее всего сходится SGD. Что касается остальных двух оптимизаторов, то при одинаковых настройках обучения быстрее сходится RMSProp